# Preprocess review.json to dictionary
## Yelp data

* The reviews that are less than 90 days before the last review date of each business are removed
* The cleaned reviews data are stored in a dictionary and dumped in review_cleaned_d.p
* Structure of the dictionary: {key:business_id, value: {key: Review Date, value: list of review text}}

1\. Read .json file into dictionary{key:business_id, value: {key: Review Date, value: list of review text}}

In [1]:
import json
import pandas as pd
import pickle

In [2]:
import pickle

reviews_dict = {}
with open("Data/yelp_academic_dataset_review.json", "r") as f:
        for line in f:
            line = json.loads(line)
            if line["business_id"] not in reviews_dict:
                reviews_sub_dict = {}
                reviews_sub_dict[line["date"]] = [line["text"]]
                reviews_dict.update({line["business_id"]:reviews_sub_dict})
            else:
                if line["date"] not in reviews_dict[line["business_id"]]:
                    reviews_dict[line["business_id"]].update({line["date"]:[line["text"]]})
                else:
                    reviews_dict[line["business_id"]][line["date"]].append(line["text"])
            
            
# #Pickle save
# with open("data/yelp_academic_dataset_review.json", "wb") as f:
#         pickle.dump(store, f)
# #Pickle load
# with open("PATH to .p file", "rb") as f:
#         store = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/yelp_academic_dataset_review.json'

In [3]:
reviews_dict["2aFiy99vNLklCx3T_tGS9A"]

{'2010-12-29': ["After being on the phone with Verizon Wireless trying to figure out why my phone wasn't working for 4.5 hours, I was put in touch with Sharpie Tech. Well, after 10 seconds they fixed the problem. As the owner of a company that does our best numbers over the holiday season, having my phone out of order for 4.5 hours was horrible. Sharpie Tech fixed the problem in record time, even Verizon was shocked. I highly recommend working with this company and I look forward to working with them more. \n\n-Rachel Charlupski\nFounder and CEO, The Babysitting Company"],
 '2011-04-29': ['Great service! Corey is very service oriented. Works fast and very effiecient with his time. Going to use him again real soon to do additional IT services. thanks Corey.'],
 '2011-10-10': ["If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I

2.Dump the dictionary into .p file. Note: use workaround for issue 24658

In [3]:
file_path = "data/yelp_review.p"

class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size

def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)
    
def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

In [3]:
# Dump the dictionary into .p file
pickle_dump(reviews_dict, file_path)

NameError: name 'reviews_dict' is not defined

3.Output dictionary from reivews.p file

In [49]:
import pickle
review_d = pickle_load(file_path)

In [54]:
review_d["--DaPTJW3-tB1vP-PfdTEg"]

{'2012-06-04': ["A super pleasant place for a delicious breakfast.\n\nThe Hubster and I came here Sunday at 1pm (peak time for the partier set) and were pleased to find no line up, no pretension and a tasty tasty breakfast at decent prices.\n\nI'm not kidding when I say this place is pleasant. It's cozy, clean and yes.. sunny. The service is very friendly and their orange juice is fresh-squeezed. It's also quiet, which is great for those who may have overdone it the night before. A standard breakfast (2 eggs, bacon/sausage/ham, toast, homefries) is about $9. Their home fries are crispy on the outside, soft and nicely spiced. \n\nDespite Boom being down the street, this is going to be our go-to breakfast spot in the neighbourhood. It's just as good without the high prices and the lineup."],
 '2012-06-14': ["This is an attractive, clean restaurant that serves a good, greasy spoon style breakfast for cheap. The waiter was very attentive and efficient, and the coffee was above average. If 

3.Remove the reviews which is less than 90 days before the last review date  

In [70]:
import time
import datetime

ThreeMonths = 7776000 # 90 days in second

def get_time(date_str):
    return time.mktime(datetime.datetime.strptime(date_str, "%Y-%m-%d").timetuple())

#input dictionary and a string of bus_id, output a date string of the last review 
def last_review_date(review_dict, bus_id):
    MIN_TIME = "2000-01-01"
    last_date = MIN_TIME
    if bus_id not in review_dict:
        return "error: not found"
    for key, value in review_dict[bus_id].items():
        if get_time(last_date) < get_time(key):
            last_date = key
    return last_date


Record the date of last review for business id "--DaPTJW3-tB1vP-PfdTEg" for later examination

In [64]:
print(last_review_date(review_d, "--DaPTJW3-tB1vP-PfdTEg"))

2017-01-08


In [68]:
import copy
review_cleaned_d = copy.deepcopy(review_d)

In [72]:
for key, value in review_cleaned_d.items():
    last_date = last_review_date(review_cleaned_d, key)
    invalid = []
    for k, v in value.items():
        if get_time(k) > get_time(last_date) - ThreeMonths:
            invalid.append(k)
    for i in invalid:
        del review_cleaned_d[key][i]

Check the reviews in 2017 have been removed.

In [73]:
review_cleaned_d["--DaPTJW3-tB1vP-PfdTEg"]

{'2012-06-04': ["A super pleasant place for a delicious breakfast.\n\nThe Hubster and I came here Sunday at 1pm (peak time for the partier set) and were pleased to find no line up, no pretension and a tasty tasty breakfast at decent prices.\n\nI'm not kidding when I say this place is pleasant. It's cozy, clean and yes.. sunny. The service is very friendly and their orange juice is fresh-squeezed. It's also quiet, which is great for those who may have overdone it the night before. A standard breakfast (2 eggs, bacon/sausage/ham, toast, homefries) is about $9. Their home fries are crispy on the outside, soft and nicely spiced. \n\nDespite Boom being down the street, this is going to be our go-to breakfast spot in the neighbourhood. It's just as good without the high prices and the lineup."],
 '2012-06-14': ["This is an attractive, clean restaurant that serves a good, greasy spoon style breakfast for cheap. The waiter was very attentive and efficient, and the coffee was above average. If 

In [75]:
pickle_dump(review_cleaned_d, "data/processed/review_cleaned_d.p")

writing total_bytes=2461331425...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 2147483648)... done.
writing bytes [2147483648, 2461331425)... done.
